<a href="https://colab.research.google.com/github/UnDauphin/Tour-mundial/blob/main/ProyectoEDD2LAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meter Datos


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

Leer Datos


In [ ]:
df=pd.read_excel("/content/sample_data/flights_final.xlsx")

In [ ]:
df

,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude
0,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
1,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834
2,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599
3,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100
4,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699
...,...,...,...,...,...,...,...,...,...,...,...,...
66925,CAN,Guangzhou Baiyun International Airport,Guangzhou,China,23.392401,113.299004,WUH,Wuhan Tianhe International Airport,Wuhan,China,30.783800,114.208000
66926,MSR,Muş Airport,Mus,Turkey,38.747799,41.661201,SAW,Sabiha Gökçen International Airport,Istanbul,Turkey,40.898602,29.309200
66927,DEN,Denver International Airport,Denver,United States,39.861698,-104.672996,OMA,Eppley Airfield,Omaha,United States,41.303200,-95.894096
66928,SHE,Taoxian Airport,Shenyang,China,41.639801,123.483002,CIF,Chifeng Airport,Chifeng,China,42.235001,118.907997


Folium

In [ ]:
m1=folium.Map(location=[0,0], tiles='cartodbpositron', zoom_start=2)
m1

In [ ]:
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic
import pandas as pd

# Crear el mapa
m1 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Función para agregar un marcador al mapa
def agregar_marcador(mapa, location, tooltip, popup, color="green"):
    folium.Marker(
        location=location,
        tooltip=tooltip,
        popup=popup,
        icon=folium.Icon(color=color),
    ).add_to(mapa)

# Función para conectar dos puntos con una línea
def conectar_puntos(mapa, loc1, loc2, color="blue"):
    folium.PolyLine(locations=[loc1, loc2], color=color, weight=2.5, opacity=1).add_to(mapa)

# Función para calcular la distancia entre dos puntos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Bucle principal
continuar = True
while continuar:
    # Pedir al usuario que ingrese el código de un aeropuerto de salida
    codigo_salida = input("Ingresa el código del aeropuerto de salida: ").upper()

    # Filtrar las filas donde el código de aeropuerto de salida coincida
    aeropuertos_salida = df[df['Source Airport Code'] == codigo_salida]

    if aeropuertos_salida.empty:
        print("No se encontraron vuelos desde este aeropuerto. Inténtalo de nuevo.")
        continue

    # Mostrar la lista de destinos posibles
    destinos_posibles = aeropuertos_salida[['Destination Airport Code', 'Destination Airport Name']].drop_duplicates()

    if destinos_posibles.empty:
        print(f"No hay destinos disponibles desde el aeropuerto {codigo_salida}.")
        continue

    # Mostrar todos los destinos posibles sin truncar
    print(f"Destinos posibles desde el aeropuerto {codigo_salida}:")
    for index, row in destinos_posibles.iterrows():
        print(f"{row['Destination Airport Code']}: {row['Destination Airport Name']}")

    # Pedir al usuario que seleccione un destino
    codigo_destino = input("Elige el código del aeropuerto de destino: ").upper()

    # Buscar las coordenadas de los aeropuertos de salida y destino
    destino_seleccionado = aeropuertos_salida[aeropuertos_salida['Destination Airport Code'] == codigo_destino]

    if destino_seleccionado.empty:
        print("Destino no válido. Inténtalo de nuevo.")
        continue

    fila_salida = aeropuertos_salida.iloc[0]  # Solo se necesita la primera fila
    fila_destino = destino_seleccionado.iloc[0]

    # Coordenadas de salida y destino
    coordenadas_salida = [fila_salida['Source Airport Latitude'], fila_salida['Source Airport Longitude']]
    coordenadas_destino = [fila_destino['Destination Airport Latitude'], fila_destino['Destination Airport Longitude']]

    # Calcular la distancia
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)
    print(f"La distancia entre los aeropuertos es {distancia:.2f} kilómetros.")

    # Añadir los puntos al mapa
    agregar_marcador(m1, coordenadas_salida, "Aeropuerto de salida", fila_salida['Source Airport Name'], color="green")
    agregar_marcador(m1, coordenadas_destino, "Aeropuerto de destino", fila_destino['Destination Airport Name'], color="blue")

    # Conectar los puntos con una línea
    conectar_puntos(m1, coordenadas_salida, coordenadas_destino)

    # Preguntar si quiere continuar
    continuar = input("¿Quieres agregar otro vuelo? (sí/no): ").lower() == "sí"

# Guardar el mapa final
m1.save("mapa_vuelos.html")


Ingresa el código del aeropuerto de salida: IKO
Destinos posibles desde el aeropuerto IKO:
DUT: Unalaska Airport
Elige el código del aeropuerto de destino: DUT
La distancia entre los aeropuertos es 186.70 kilómetros.
¿Quieres agregar otro vuelo? (sí/no): Sí
Ingresa el código del aeropuerto de salida: DUT
Destinos posibles desde el aeropuerto DUT:
AKB: Atka Airport
KQA: Akutan Seaplane Base
IKO: Nikolski Air Station
Elige el código del aeropuerto de destino: AKB
La distancia entre los aeropuertos es 546.31 kilómetros.
¿Quieres agregar otro vuelo? (sí/no): no


In [ ]:
m1

Intalar Geopy

In [ ]:
pip install geopy

Punto 1

In [ ]:
from geopy.distance import geodesic
import matplotlib.pyplot as plt

# Crear un grafo vacío manual como un diccionario de listas (grafo de adyacencia)
G_manual = {}

# Función para calcular la distancia entre dos puntos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Función para agregar un nodo y una arista al grafo manual
def agregar_arista(grafo, nodo1, nodo2):
    if nodo1 not in grafo:
        grafo[nodo1] = []
    if nodo2 not in grafo:
        grafo[nodo2] = []
    grafo[nodo1].append(nodo2)
    grafo[nodo2].append(nodo1)

# Construir el grafo manualmente (iterando por el dataframe)
for index, row in df.iterrows():
    # Coordenadas de salida y destino
    coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
    coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]

    # Códigos de los aeropuertos de salida y destino
    aeropuerto_salida = row['Source Airport Code']
    aeropuerto_destino = row['Destination Airport Code']

    # Calcular la distancia entre los aeropuertos (opcional, guardado en otro lado)
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)

    # Agregar los aeropuertos al grafo manual con una arista entre ellos
    agregar_arista(G_manual, aeropuerto_salida, aeropuerto_destino)

# Función DFS para encontrar todos los nodos conectados a un nodo dado
def dfs(grafo, nodo, visitado, componente):
    visitado.add(nodo)
    componente.append(nodo)
    for vecino in grafo[nodo]:
        if vecino not in visitado:
            dfs(grafo, vecino, visitado, componente)

# Función para encontrar todas las componentes conexas
def encontrar_componentes_conexas(grafo):
    visitado = set()
    componentes_conexas = []

    # Recorrer todos los nodos en el grafo
    for nodo in grafo:
        if nodo not in visitado:
            componente = []
            dfs(grafo, nodo, visitado, componente)
            componentes_conexas.append(componente)

    return componentes_conexas

# Obtener las componentes conexas del grafo manual
componentes_conexas = encontrar_componentes_conexas(G_manual)

# Imprimir el número total de componentes conexas
print(f"Total de componentes conexas: {len(componentes_conexas)}")

# Imprimir el tamaño (número de vértices) de cada componente conexa
for i, componente in enumerate(componentes_conexas, 1):
    print(f"Componente conexa {i}: {len(componente)} vértices")

# Configuración de visualización usando Matplotlib para graficar el grafo
plt.figure(figsize=(20, 15))  # Ajustar el tamaño de la figura
nodos = list(G_manual.keys())

# Crear posiciones aleatorias para los nodos (no hay posiciones reales ya que es manual)
posiciones_manual = {nodo: (i % 10, i // 10) for i, nodo in enumerate(nodos)}





Total de componentes conexas: 7
Componente conexa 1: 3230 vértices
Componente conexa 2: 4 vértices
Componente conexa 3: 4 vértices
Componente conexa 4: 10 vértices
Componente conexa 5: 4 vértices
Componente conexa 6: 2 vértices
Componente conexa 7: 2 vértices


<Figure size 2000x1500 with 0 Axes>

In [ ]:
#intento 2

import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

# Crear el mapa inicial centrado en una ubicación genérica
m1 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Set para almacenar los aeropuertos únicos (vértices y ubicaciones)
vertices = set()  # Guarda códigos de aeropuertos únicos
marcadores_agregados = {}  # Guarda las coordenadas de los aeropuertos que ya tienen marcador

# Crear un MarkerCluster para agrupar puntos cercanos
marker_cluster = MarkerCluster().add_to(m1)

# Función para agregar un marcador al cluster (solo si no ha sido agregado previamente)
def agregar_marcador_si_no_existe(cluster, location, codigo_aeropuerto):
    if codigo_aeropuerto not in marcadores_agregados:
        folium.Marker(
            location=location,
            tooltip=codigo_aeropuerto,
            icon=folium.Icon(color="green"),
        ).add_to(cluster)
        # Guardar que este aeropuerto ya tiene un marcador
        marcadores_agregados[codigo_aeropuerto] = location

# Función para conectar dos puntos con una línea (sin popups)
def conectar_puntos(mapa, loc1, loc2, color="blue"):
    folium.PolyLine(locations=[loc1, loc2], color=color, weight=1.5, opacity=0.7).add_to(mapa)

# Función para calcular la distancia entre dos puntos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Iterar por cada fila del dataframe para obtener los aeropuertos de salida y destino
for index, row in df.iterrows():
    # Coordenadas de salida y destino
    coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
    coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]

    # Aeropuertos como vértices
    vertices.add(row['Source Airport Code'])  # Aeropuerto de salida
    vertices.add(row['Destination Airport Code'])  # Aeropuerto de destino

    # Calcular la distancia
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)

    # Añadir los puntos al cluster solo si no han sido añadidos previamente
    agregar_marcador_si_no_existe(marker_cluster, coordenadas_salida, row['Source Airport Code'])
    agregar_marcador_si_no_existe(marker_cluster, coordenadas_destino, row['Destination Airport Code'])

    # Conectar los puntos con una línea en el mapa
    conectar_puntos(m1, coordenadas_salida, coordenadas_destino)

# Imprimir el número total de vértices (aeropuertos únicos)
print(f"\nTotal de vértices (aeropuertos únicos): {len(vertices)}")

# Guardar el mapa con todas las conexiones
m1.save("mapa_vuelos_optimizado_sin_duplicados.html")



Total de vértices (aeropuertos únicos): 3256


Punto 2

In [ ]:
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Crear el mapa inicial centrado en una ubicación genérica
m1 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Set para almacenar los aeropuertos únicos (vértices y ubicaciones)
marcadores_agregados = {}  # Guarda las coordenadas de los aeropuertos que ya tienen marcador

# Crear un MarkerCluster para agrupar puntos cercanos
marker_cluster = MarkerCluster().add_to(m1)

# Función para agregar un marcador al cluster (solo si no ha sido agregado previamente)
def agregar_marcador_si_no_existe(cluster, location, codigo_aeropuerto):
    if codigo_aeropuerto not in marcadores_agregados:
        folium.Marker(
            location=location,
            tooltip=codigo_aeropuerto,
            icon=folium.Icon(color="green"),
        ).add_to(cluster)
        # Guardar que este aeropuerto ya tiene un marcador
        marcadores_agregados[codigo_aeropuerto] = location

# Función para conectar dos puntos con una línea (sin popups)
def conectar_puntos(mapa, loc1, loc2, color="blue"):
    folium.PolyLine(locations=[loc1, loc2], color=color, weight=2.5, opacity=1).add_to(mapa)

# Función para calcular la distancia entre dos puntos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Función para encontrar el conjunto al que pertenece un nodo
def find(parent, i):
    if parent[i] == i:
        return i
    else:
        return find(parent, parent[i])

# Función para unir dos subconjuntos (union by rank)
def union(parent, rank, x, y):
    root_x = find(parent, x)
    root_y = find(parent, y)

    # Unir los árboles de menor rango bajo el de mayor rango
    if rank[root_x] < rank[root_y]:
        parent[root_x] = root_y
    elif rank[root_x] > rank[root_y]:
        parent[root_y] = root_x
    else:
        parent[root_y] = root_x
        rank[root_x] += 1

# Función que implementa el algoritmo de Kruskal
def kruskal_minimum_spanning_tree(df):
    # Lista para almacenar el árbol de expansión mínimo (MST)
    mst = []

    # Inicializar padres y rangos para la estructura union-find
    parent = {}
    rank = {}

    # Inicializar cada nodo (aeropuerto) como su propio conjunto
    for index, row in df.iterrows():
        parent[row['Source Airport Code']] = row['Source Airport Code']
        parent[row['Destination Airport Code']] = row['Destination Airport Code']
        rank[row['Source Airport Code']] = 0
        rank[row['Destination Airport Code']] = 0

    # Obtener todas las aristas del grafo junto con sus pesos (distancia entre aeropuertos)
    edges = []
    for index, row in df.iterrows():
        coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
        coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]
        distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)
        edges.append((row['Source Airport Code'], row['Destination Airport Code'], distancia))

    # Ordenar las aristas por peso (de menor a mayor)
    edges.sort(key=lambda x: x[2])

    # Recorrer las aristas ordenadas
    for u, v, weight in edges:
        root_u = find(parent, u)
        root_v = find(parent, v)

        # Si los vértices u y v no forman un ciclo, los agregamos al MST
        if root_u != root_v:
            mst.append((u, v, weight))
            union(parent, rank, root_u, root_v)

    return mst

# Función para encontrar las componentes conexas a partir del MST
def encontrar_componentes_conexas(mst):
    componentes = []
    visitado = set()

    def dfs(nodo, componente):
        visitado.add(nodo)
        componente.append(nodo)
        for u, v, _ in mst:
            if u == nodo and v not in visitado:
                dfs(v, componente)
            elif v == nodo and u not in visitado:
                dfs(u, componente)

    for u, v, _ in mst:
        if u not in visitado:
            componente = []
            dfs(u, componente)
            componentes.append(componente)
        if v not in visitado:
            componente = []
            dfs(v, componente)
            componentes.append(componente)

    return componentes

# Aplicar el algoritmo de Kruskal al dataframe de vuelos
mst = kruskal_minimum_spanning_tree(df)

# Obtener las componentes conexas del MST
componentes_conexas = encontrar_componentes_conexas(mst)

# Colormap para asignar diferentes colores a las componentes
colormap = plt.cm.get_cmap('tab10', len(componentes_conexas))
colores_componentes = [colormap(i) for i in range(len(componentes_conexas))]

# Calcular el peso total del MST
peso_total_mst = sum([weight for u, v, weight in mst])

print(f"\nPeso total del árbol de expansión mínima (MST): {peso_total_mst:.2f} km")

# Peso de cada componente (corregido)
peso_componentes = []
for componente in componentes_conexas:
    peso_componente = 0
    aristas_visitadas = set()  # Conjunto para almacenar las aristas ya contadas

    for nodo in componente:
        for u, v, weight in mst:
            # Crear una representación única para cada arista (sin importar el orden de u y v)
            arista = tuple(sorted([u, v]))

            # Solo sumar el peso si la arista no ha sido visitada
            if arista not in aristas_visitadas and (u in componente and v in componente):
                peso_componente += weight
                aristas_visitadas.add(arista)  # Marcar la arista como visitada

    peso_componentes.append(peso_componente)

# Visualización del MST en el mapa
for idx, componente in enumerate(componentes_conexas):
    color_rgb = colores_componentes[idx]
    color_hex = '#{:02x}{:02x}{:02x}'.format(int(color_rgb[0]*255), int(color_rgb[1]*255), int(color_rgb[2]*255))

    for nodo in componente:
        for u, v, weight in mst:
            if (u == nodo or v == nodo) and (u in componente and v in componente):
                fila_salida = df[df['Source Airport Code'] == u].iloc[0]
                fila_destino = df[df['Destination Airport Code'] == v].iloc[0]

                coordenadas_salida = [fila_salida['Source Airport Latitude'], fila_salida['Source Airport Longitude']]
                coordenadas_destino = [fila_destino['Destination Airport Latitude'], fila_destino['Destination Airport Longitude']]

                # Añadir los puntos al cluster solo si no han sido añadidos previamente
                agregar_marcador_si_no_existe(marker_cluster, coordenadas_salida, u)
                agregar_marcador_si_no_existe(marker_cluster, coordenadas_destino, v)

                # Conectar los puntos con una línea en el mapa, cada componente con un color único
                conectar_puntos(m1, coordenadas_salida, coordenadas_destino, color=color_hex)

# Guardar el mapa con el árbol de expansión mínima (MST)
m1.save("mapa_vuelos_mst_componentes.html")

# Número total de vértices y aristas en el MST
print(f"\nTotal de aeropuertos (vértices) en el MST: {len(set([u for u, v, w in mst] + [v for u, v, w in mst]))}")
print(f"Total de conexiones (aristas) en el MST: {len(mst)}")

# Mostrar el peso de cada componente
for idx, peso in enumerate(peso_componentes):
    print(f"Peso del árbol de expansión mínima de la componente {idx + 1}: {peso:.2f} km")


<ipython-input-17-a68beb6c6c99>:129: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = plt.cm.get_cmap('tab10', len(componentes_conexas))



Peso total del árbol de expansión mínima (MST): 1201818.14 km

Total de aeropuertos (vértices) en el MST: 3256
Total de conexiones (aristas) en el MST: 3249
Peso del árbol de expansión mínima de la componente 1: 1197600.62 km
Peso del árbol de expansión mínima de la componente 2: 256.23 km
Peso del árbol de expansión mínima de la componente 3: 37.97 km
Peso del árbol de expansión mínima de la componente 4: 789.50 km
Peso del árbol de expansión mínima de la componente 5: 1439.46 km
Peso del árbol de expansión mínima de la componente 6: 98.81 km
Peso del árbol de expansión mínima de la componente 7: 1595.55 km


PUNTO 3 Y 4

In [ ]:
import folium
from geopy.distance import geodesic
import math

# Crear un mapa vacío
m2 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Función para calcular la distancia entre dos puntos geográficos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Función para mostrar la información de un aeropuerto específico
def mostrar_informacion_aeropuerto(aeropuerto, airport_data):
    info = airport_data[aeropuerto]
    print(f"Información del aeropuerto {aeropuerto}:")
    print(f"Nombre: {info['name']}")
    print(f"Ciudad: {info['city']}")
    print(f"País: {info['country']}")
    print(f"Latitud: {info['latitude']}")
    print(f"Longitud: {info['longitude']}")
    print()

# Implementación del algoritmo de Dijkstra
def dijkstra(grafo, inicio):
    distancias = {nodo: math.inf for nodo in grafo}  # Asignar infinito a todas las distancias
    distancias[inicio] = 0  # La distancia del nodo de inicio a sí mismo es 0
    padres = {nodo: None for nodo in grafo}  # Almacenar el predecesor de cada nodo
    visitados = set()

    while len(visitados) < len(grafo):
        # Escoger el nodo con la menor distancia que no haya sido visitado
        nodo_actual = None
        for nodo in grafo:
            if nodo not in visitados:
                if nodo_actual is None or distancias[nodo] < distancias[nodo_actual]:
                    nodo_actual = nodo

        # Si no se puede avanzar (todos los nodos restantes son inalcanzables)
        if distancias[nodo_actual] == math.inf:
            break

        # Marcar el nodo como visitado
        visitados.add(nodo_actual)

        # Revisar las conexiones del nodo actual
        for vecino, peso in grafo[nodo_actual]['conexiones']:
            nueva_distancia = distancias[nodo_actual] + peso
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia
                padres[vecino] = nodo_actual

    return distancias, padres

# Función para reconstruir el camino desde el destino hasta el inicio
def reconstruir_camino(padres, destino):
    camino = []
    nodo = destino
    while nodo is not None:
        camino.insert(0, nodo)
        nodo = padres[nodo]
    return camino

# Función para obtener los 10 caminos mínimos más largos
def obtener_top_10_caminos_minimos_mas_largos(aeropuerto_inicio, grafo, airport_data):
    distancias, padres = dijkstra(grafo, aeropuerto_inicio)

    # Filtrar distancias válidas (aquellas que no son infinitas)
    distancias_validas = [(nodo, distancia) for nodo, distancia in distancias.items() if distancia != math.inf]

    # Ordenar las distancias de mayor a menor
    distancias_validas.sort(key=lambda x: x[1], reverse=True)

    # Obtener los 10 caminos mínimos más largos
    top_10_largos = distancias_validas[:10]

    # Mostrar la información de los 10 aeropuertos con caminos mínimos más largos
    print(f"Top 10 caminos mínimos más largos desde {aeropuerto_inicio}:")
    for aeropuerto_destino, distancia in top_10_largos:
        camino = reconstruir_camino(padres, aeropuerto_destino)
        info = airport_data[aeropuerto_destino]
        print(f"Camino: {' -> '.join(camino)}")
        print(f"{aeropuerto_destino}: {info['name']}, {info['city']}, {info['country']}")
        print(f"Distancia: {distancia:.2f} km")
        print(f"Latitud: {info['latitude']}, Longitud: {info['longitude']}")
        print()

    return top_10_largos, padres

# Función para dibujar los 10 caminos mínimos más largos en el mapa
def dibujar_top_10_caminos_minimos(mapa, aeropuerto_inicio, top_10_largos, grafo, padres):
    # Dibujar el aeropuerto de inicio
    coordenadas_inicio = grafo[aeropuerto_inicio]['pos']
    folium.Marker(coordenadas_inicio, tooltip=aeropuerto_inicio, icon=folium.Icon(color="blue")).add_to(mapa)

    # Dibujar los caminos mínimos más largos
    for aeropuerto_destino, _ in top_10_largos:
        camino = reconstruir_camino(padres, aeropuerto_destino)

        # Dibujar las líneas de conexión en el mapa para cada segmento del camino mínimo
        for i in range(len(camino) - 1):
            coordenadas_salida = grafo[camino[i]]['pos']
            coordenadas_destino = grafo[camino[i+1]]['pos']

            folium.PolyLine(locations=[coordenadas_salida, coordenadas_destino], color="red", weight=3).add_to(mapa)

        # Marcar el aeropuerto de destino en el mapa
        coordenadas_destino = grafo[aeropuerto_destino]['pos']
        folium.Marker(coordenadas_destino, tooltip=aeropuerto_destino, icon=folium.Icon(color="green")).add_to(mapa)

# Implementación del algoritmo de Dijkstra
def dijkstra1(grafo, inicio, destino):
    distancias = {nodo: math.inf for nodo in grafo}  # Asignar infinito a todas las distancias
    distancias[inicio] = 0  # La distancia del nodo de inicio a sí mismo es 0
    padres = {nodo: None for nodo in grafo}  # Almacenar el predecesor de cada nodo
    visitados = set()

    while len(visitados) < len(grafo):
        # Escoger el nodo con la menor distancia que no haya sido visitado
        nodo_actual = None
        for nodo in grafo:
            if nodo not in visitados:
                if nodo_actual is None or distancias[nodo] < distancias[nodo_actual]:
                    nodo_actual = nodo

        # Si no se puede avanzar (todos los nodos restantes son inalcanzables)
        if distancias[nodo_actual] == math.inf:
            break

        # Marcar el nodo como visitado
        visitados.add(nodo_actual)

        # Revisar las conexiones del nodo actual
        for vecino, peso in grafo[nodo_actual]['conexiones']:
            nueva_distancia = distancias[nodo_actual] + peso
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia
                padres[vecino] = nodo_actual

    # Reconstruir el camino
    camino = []
    nodo = destino
    while nodo is not None:
        camino.insert(0, nodo)
        nodo = padres[nodo]

    return camino, distancias[destino]

# Función para dibujar el camino mínimo en el mapa
def dibujar_camino_mapa(mapa, grafo, camino):
    for i in range(len(camino) - 1):
        aeropuerto_salida = camino[i]
        aeropuerto_destino = camino[i+1]
        coordenadas_salida = grafo[aeropuerto_salida]['pos']
        coordenadas_destino = grafo[aeropuerto_destino]['pos']

        # Dibujar la línea en el mapa
        folium.PolyLine(locations=[coordenadas_salida, coordenadas_destino], color="red", weight=3).add_to(mapa)

        # Marcar los aeropuertos en el mapa
        folium.Marker(coordenadas_salida, tooltip=aeropuerto_salida, icon=folium.Icon(color="blue")).add_to(mapa)
        folium.Marker(coordenadas_destino, tooltip=aeropuerto_destino, icon=folium.Icon(color="green")).add_to(mapa)


# Crear el grafo de aeropuertos
grafo = {}
airport_data = {}

for index, row in df.iterrows():
    aeropuerto_salida = row['Source Airport Code']
    aeropuerto_destino = row['Destination Airport Code']
    coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
    coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]

    # Calcular la distancia
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)

    # Agregar nodos y aristas al grafo
    if aeropuerto_salida not in grafo:
        grafo[aeropuerto_salida] = {'pos': coordenadas_salida, 'conexiones': []}
    if aeropuerto_destino not in grafo:
        grafo[aeropuerto_destino] = {'pos': coordenadas_destino, 'conexiones': []}

    grafo[aeropuerto_salida]['conexiones'].append((aeropuerto_destino, distancia))
    grafo[aeropuerto_destino]['conexiones'].append((aeropuerto_salida, distancia))

    # Almacenar la información del aeropuerto
    airport_data[aeropuerto_salida] = {
        'name': row['Source Airport Name'],
        'city': row['Source Airport City'],
        'country': row['Source Airport Country'],
        'latitude': row['Source Airport Latitude'],
        'longitude': row['Source Airport Longitude']
    }
    airport_data[aeropuerto_destino] = {
        'name': row['Destination Airport Name'],
        'city': row['Destination Airport City'],
        'country': row['Destination Airport Country'],
        'latitude': row['Destination Airport Latitude'],
        'longitude': row['Destination Airport Longitude']
    }

# Pedir al usuario que ingrese un aeropuerto
aeropuerto_inicio = input("Ingrese el código del aeropuerto: ")
print("Escriba 1 para cambiar el codigo")
print("Escriba 2 para mostrar informacion del codigo")
print("Escriba 3 para mostrar los 10 caminos minimos mas largos")
print("Escriba 4 para dar un segundo codigo y hallar el camino minimo entre estos dos")
print("Escriba 5 para salir")
while(True):
  menu = input("Indique que quiere hacer con el codigo: ")
  if menu == "1":
    aeropuerto_inicio = input("Ingrese el código del aeropuerto: ")
  elif menu == "2":
    mostrar_informacion_aeropuerto(aeropuerto_inicio, airport_data)
  elif menu == "3":
    top_10_largos, padres = obtener_top_10_caminos_minimos_mas_largos(aeropuerto_inicio, grafo, airport_data)
    m2 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)
    dibujar_top_10_caminos_minimos(m2, aeropuerto_inicio, top_10_largos, grafo, padres)
    m2.save("mapa_top_10_caminos_minimos_mas_largos.html")
  elif menu == "4":
    aeropuerto_destino = input("Ingrese el código del aeropuerto destino: ")
    camino_minimo, distancia_total = dijkstra1(grafo, aeropuerto_inicio, aeropuerto_destino)
    print(f"El camino mínimo desde {aeropuerto_inicio} hasta {aeropuerto_destino} es:")
    print(" -> ".join(camino_minimo))
    print(f"Distancia total: {distancia_total:.2f} km")
    for aeropuerto in camino_minimo:
      info = airport_data[aeropuerto]
      print(f"{aeropuerto}: {info['name']}, {info['city']}, {info['country']}, Lat: {info['latitude']}, Lon: {info['longitude']}")
    m2 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)
    dibujar_camino_mapa(m2, grafo, camino_minimo)
    m2.save("mapa_camino_minimo.html")
  elif menu == "5":
    break
  else:
    print("Opcion invalida")

Punto 3 por separado

In [ ]:
import folium
from geopy.distance import geodesic
import math

# Crear un mapa vacío
m2 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Función para calcular la distancia entre dos puntos geográficos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Función para mostrar la información de un aeropuerto específico
def mostrar_informacion_aeropuerto(aeropuerto, airport_data):
    info = airport_data[aeropuerto]
    print(f"Información del aeropuerto {aeropuerto}:")
    print(f"Nombre: {info['name']}")
    print(f"Ciudad: {info['city']}")
    print(f"País: {info['country']}")
    print(f"Latitud: {info['latitude']}")
    print(f"Longitud: {info['longitude']}")
    print()

# Implementación del algoritmo de Dijkstra
def dijkstra(grafo, inicio):
    distancias = {nodo: math.inf for nodo in grafo}  # Asignar infinito a todas las distancias
    distancias[inicio] = 0  # La distancia del nodo de inicio a sí mismo es 0
    padres = {nodo: None for nodo in grafo}  # Almacenar el predecesor de cada nodo
    visitados = set()

    while len(visitados) < len(grafo):
        # Escoger el nodo con la menor distancia que no haya sido visitado
        nodo_actual = None
        for nodo in grafo:
            if nodo not in visitados:
                if nodo_actual is None or distancias[nodo] < distancias[nodo_actual]:
                    nodo_actual = nodo

        # Si no se puede avanzar (todos los nodos restantes son inalcanzables)
        if distancias[nodo_actual] == math.inf:
            break

        # Marcar el nodo como visitado
        visitados.add(nodo_actual)

        # Revisar las conexiones del nodo actual
        for vecino, peso in grafo[nodo_actual]['conexiones']:
            nueva_distancia = distancias[nodo_actual] + peso
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia
                padres[vecino] = nodo_actual

    return distancias, padres

# Función para reconstruir el camino desde el destino hasta el inicio
def reconstruir_camino(padres, destino):
    camino = []
    nodo = destino
    while nodo is not None:
        camino.insert(0, nodo)
        nodo = padres[nodo]
    return camino

# Función para obtener los 10 caminos mínimos más largos
def obtener_top_10_caminos_minimos_mas_largos(aeropuerto_inicio, grafo, airport_data):
    distancias, padres = dijkstra(grafo, aeropuerto_inicio)

    # Filtrar distancias válidas (aquellas que no son infinitas)
    distancias_validas = [(nodo, distancia) for nodo, distancia in distancias.items() if distancia != math.inf]

    # Ordenar las distancias de mayor a menor
    distancias_validas.sort(key=lambda x: x[1], reverse=True)

    # Obtener los 10 caminos mínimos más largos
    top_10_largos = distancias_validas[:10]

    # Mostrar la información de los 10 aeropuertos con caminos mínimos más largos
    print(f"Top 10 caminos mínimos más largos desde {aeropuerto_inicio}:")
    for aeropuerto_destino, distancia in top_10_largos:
        camino = reconstruir_camino(padres, aeropuerto_destino)
        info = airport_data[aeropuerto_destino]
        print(f"Camino: {' -> '.join(camino)}")
        print(f"{aeropuerto_destino}: {info['name']}, {info['city']}, {info['country']}")
        print(f"Distancia: {distancia:.2f} km")
        print(f"Latitud: {info['latitude']}, Longitud: {info['longitude']}")
        print()

    return top_10_largos, padres

# Función para dibujar los 10 caminos mínimos más largos en el mapa
def dibujar_top_10_caminos_minimos(mapa, aeropuerto_inicio, top_10_largos, grafo, padres):
    # Dibujar el aeropuerto de inicio
    coordenadas_inicio = grafo[aeropuerto_inicio]['pos']
    folium.Marker(coordenadas_inicio, tooltip=aeropuerto_inicio, icon=folium.Icon(color="blue")).add_to(mapa)

    # Dibujar los caminos mínimos más largos
    for aeropuerto_destino, _ in top_10_largos:
        camino = reconstruir_camino(padres, aeropuerto_destino)

        # Dibujar las líneas de conexión en el mapa para cada segmento del camino mínimo
        for i in range(len(camino) - 1):
            coordenadas_salida = grafo[camino[i]]['pos']
            coordenadas_destino = grafo[camino[i+1]]['pos']

            folium.PolyLine(locations=[coordenadas_salida, coordenadas_destino], color="red", weight=3).add_to(mapa)

        # Marcar el aeropuerto de destino en el mapa
        coordenadas_destino = grafo[aeropuerto_destino]['pos']
        folium.Marker(coordenadas_destino, tooltip=aeropuerto_destino, icon=folium.Icon(color="green")).add_to(mapa)

# Crear el grafo de aeropuertos
grafo = {}
airport_data = {}

for index, row in df.iterrows():
    aeropuerto_salida = row['Source Airport Code']
    aeropuerto_destino = row['Destination Airport Code']
    coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
    coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]

    # Calcular la distancia
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)

    # Agregar nodos y aristas al grafo
    if aeropuerto_salida not in grafo:
        grafo[aeropuerto_salida] = {'pos': coordenadas_salida, 'conexiones': []}
    if aeropuerto_destino not in grafo:
        grafo[aeropuerto_destino] = {'pos': coordenadas_destino, 'conexiones': []}

    grafo[aeropuerto_salida]['conexiones'].append((aeropuerto_destino, distancia))
    grafo[aeropuerto_destino]['conexiones'].append((aeropuerto_salida, distancia))

    # Almacenar la información del aeropuerto
    airport_data[aeropuerto_salida] = {
        'name': row['Source Airport Name'],
        'city': row['Source Airport City'],
        'country': row['Source Airport Country'],
        'latitude': row['Source Airport Latitude'],
        'longitude': row['Source Airport Longitude']
    }
    airport_data[aeropuerto_destino] = {
        'name': row['Destination Airport Name'],
        'city': row['Destination Airport City'],
        'country': row['Destination Airport Country'],
        'latitude': row['Destination Airport Latitude'],
        'longitude': row['Destination Airport Longitude']
    }

# Pedir al usuario que ingrese un aeropuerto
aeropuerto_inicio = input("Ingrese el código del aeropuerto: ")

# Mostrar la información del aeropuerto
mostrar_informacion_aeropuerto(aeropuerto_inicio, airport_data)

# Obtener y mostrar los 10 caminos mínimos más largos
top_10_largos, padres = obtener_top_10_caminos_minimos_mas_largos(aeropuerto_inicio, grafo, airport_data)

# Dibujar los 10 caminos mínimos más largos en el mapa
dibujar_top_10_caminos_minimos(m2, aeropuerto_inicio, top_10_largos, grafo, padres)

# Guardar el mapa con los caminos más largos
m2.save("mapa_top_10_caminos_minimos_mas_largos.html")


Ingrese el código del aeropuerto: GIG
Información del aeropuerto GIG:
Nombre: Rio Galeão – Tom Jobim International Airport
Ciudad: Rio De Janeiro
País: Brazil
Latitud: -22.80999947
Longitud: -43.25055695

Top 10 caminos mínimos más largos desde GIG:
Camino: GIG -> GRU -> JNB -> SIN -> MDC -> SOQ -> MKW -> DJJ -> MKQ
MKQ: Mopah Airport, Merauke, Indonesia
Distancia: 21245.00 km
Latitud: -8.520290375, Longitud: 140.4179993

Camino: GIG -> GRU -> JNB -> SIN -> MDC -> SOQ -> MKW -> DJJ -> OKL
OKL: Oksibil Airport, Oksibil, Indonesia
Distancia: 20845.62 km
Latitud: -4.9071, Longitud: 140.6277

Camino: GIG -> GRU -> JNB -> SIN -> MDC -> SOQ -> MKW -> DJJ -> WMX
WMX: Wamena Airport, Wamena, Indonesia
Distancia: 20829.47 km
Latitud: -4.10251, Longitud: 138.957001

Camino: GIG -> GRU -> JNB -> SIN -> MDC -> SOQ -> MKW -> DJJ
DJJ: Sentani International Airport, Jayapura, Indonesia
Distancia: 20587.66 km
Latitud: -2.576950073, Longitud: 140.5160065

Camino: GIG -> SCL -> AKL -> CNS -> GUM -> YAP


Punto 4 por separado

In [ ]:
import folium
from geopy.distance import geodesic
import math

# Crear un mapa vacío
m2 = folium.Map(location=[0, 0], tiles='cartodbpositron', zoom_start=2)

# Función para calcular la distancia entre dos puntos geográficos
def calcular_distancia(loc1, loc2):
    return geodesic(loc1, loc2).kilometers

# Implementación del algoritmo de Dijkstra
def dijkstra(grafo, inicio, destino):
    distancias = {nodo: math.inf for nodo in grafo}  # Asignar infinito a todas las distancias
    distancias[inicio] = 0  # La distancia del nodo de inicio a sí mismo es 0
    padres = {nodo: None for nodo in grafo}  # Almacenar el predecesor de cada nodo
    visitados = set()

    while len(visitados) < len(grafo):
        # Escoger el nodo con la menor distancia que no haya sido visitado
        nodo_actual = None
        for nodo in grafo:
            if nodo not in visitados:
                if nodo_actual is None or distancias[nodo] < distancias[nodo_actual]:
                    nodo_actual = nodo

        # Si no se puede avanzar (todos los nodos restantes son inalcanzables)
        if distancias[nodo_actual] == math.inf:
            break

        # Marcar el nodo como visitado
        visitados.add(nodo_actual)

        # Revisar las conexiones del nodo actual
        for vecino, peso in grafo[nodo_actual]['conexiones']:
            nueva_distancia = distancias[nodo_actual] + peso
            if nueva_distancia < distancias[vecino]:
                distancias[vecino] = nueva_distancia
                padres[vecino] = nodo_actual

    # Reconstruir el camino
    camino = []
    nodo = destino
    while nodo is not None:
        camino.insert(0, nodo)
        nodo = padres[nodo]

    return camino, distancias[destino]

# Función para dibujar el camino mínimo en el mapa
def dibujar_camino_mapa(mapa, grafo, camino):
    for i in range(len(camino) - 1):
        aeropuerto_salida = camino[i]
        aeropuerto_destino = camino[i+1]
        coordenadas_salida = grafo[aeropuerto_salida]['pos']
        coordenadas_destino = grafo[aeropuerto_destino]['pos']

        # Dibujar la línea en el mapa
        folium.PolyLine(locations=[coordenadas_salida, coordenadas_destino], color="red", weight=3).add_to(mapa)

        # Marcar los aeropuertos en el mapa
        folium.Marker(coordenadas_salida, tooltip=aeropuerto_salida, icon=folium.Icon(color="blue")).add_to(mapa)
        folium.Marker(coordenadas_destino, tooltip=aeropuerto_destino, icon=folium.Icon(color="green")).add_to(mapa)

# Crear el grafo de aeropuertos
grafo = {}
airport_data = {}

for index, row in df.iterrows():
    aeropuerto_salida = row['Source Airport Code']
    aeropuerto_destino = row['Destination Airport Code']
    coordenadas_salida = [row['Source Airport Latitude'], row['Source Airport Longitude']]
    coordenadas_destino = [row['Destination Airport Latitude'], row['Destination Airport Longitude']]

    # Calcular la distancia
    distancia = calcular_distancia(coordenadas_salida, coordenadas_destino)

    # Agregar nodos y aristas al grafo
    if aeropuerto_salida not in grafo:
        grafo[aeropuerto_salida] = {'pos': coordenadas_salida, 'conexiones': []}
    if aeropuerto_destino not in grafo:
        grafo[aeropuerto_destino] = {'pos': coordenadas_destino, 'conexiones': []}

    grafo[aeropuerto_salida]['conexiones'].append((aeropuerto_destino, distancia))
    grafo[aeropuerto_destino]['conexiones'].append((aeropuerto_salida, distancia))

    # Almacenar la información del aeropuerto
    airport_data[aeropuerto_salida] = {
        'name': row['Source Airport Name'],
        'city': row['Source Airport City'],
        'country': row['Source Airport Country'],
        'latitude': row['Source Airport Latitude'],
        'longitude': row['Source Airport Longitude']
    }

# Pedir al usuario que ingrese los dos aeropuertos
aeropuerto_inicio = input("Ingrese el código del aeropuerto de inicio: ")
aeropuerto_destino = input("Ingrese el código del aeropuerto de destino: ")

# Calcular el camino mínimo utilizando Dijkstra
camino_minimo, distancia_total = dijkstra(grafo, aeropuerto_inicio, aeropuerto_destino)

# Mostrar el resultado en consola
print(f"El camino mínimo desde {aeropuerto_inicio} hasta {aeropuerto_destino} es:")
print(" -> ".join(camino_minimo))
print(f"Distancia total: {distancia_total:.2f} km")

# Mostrar información detallada de cada aeropuerto en el camino
for aeropuerto in camino_minimo:
    info = airport_data[aeropuerto]
    print(f"{aeropuerto}: {info['name']}, {info['city']}, {info['country']}, Lat: {info['latitude']}, Lon: {info['longitude']}")

# Dibujar el camino mínimo en el mapa
dibujar_camino_mapa(m2, grafo, camino_minimo)

# Guardar el mapa con el camino mínimo
m2.save("mapa_camino_minimo.html")


Ingrese el código del aeropuerto de inicio: WUH
Ingrese el código del aeropuerto de destino: IKO
El camino mínimo desde WUH hasta IKO es:
IKO
Distancia total: inf km
IKO: Nikolski Air Station, Nikolski, United States, Lat: 52.9416008, Lon: -168.848999


In [ ]:
from typing import List
class Graph:
    def __init__(self):
        self.L = {}
        self.chars = {}
        self.edges = []

    def add_node(self, code: str, name: str, city: str, country: str, lat: float, lon: float) -> None:
        self.L[code] = {}
        self.chars[code] = {"name" : name, "city": city, "country" : country, "lat" : lat, "lon" : lon}

    def add_edge(self, i: int) -> None:
        u = df["Source Airport Code"][i]
        v = df["Destination Airport Code"][i]
        if u not in self.L:
          #aqui hay que agregar el nodo con la informacion
          self.add_node(u, df["Source Airport Name"][i], df["Source Airport City"][i], df["Source Airport Country"][i], df["Source Airport Latitude"][i], df["Source Airport Longitude"][i])
        if v not in self.L:
          self.add_node(v, df["Destination Airport Name"][i], df["Destination Airport City"][i], df["Destination Airport Country"][i], df["Destination Airport Latitude"][i], df["Destination Airport Longitude"][i])
        #nose calcular la distancia
        dist = 0
        #self.L[u].append([dist, v])
        #self.L[v].append([dist, u])
        self.L[u][v] = dist
        self.L[v][u] = dist
        #self.edges.append([u, v])

    def make_graph(self):
        for i in range(len(df)):
          self.add_edge(i)

    def BFS(self, u: str) -> List[str]:
        bfs = []
        queue = []
        visit = {}
        for j in self.L:
            visit[j] = False
        visit[u] = True
        queue.append(u)
        while len(queue) > 0:
            u = queue.pop(0)
            bfs.append(u)
            for v in self.L[u]:
                if not visit[v]:
                    visit[v] = True
                    queue.append(v)
        return bfs

    def number_of_components(self) -> List[int]:
        comps = []
        gvisit = {}
        for u in self.L:
            gvisit[u] = False
        count = 0
        for u in self.L:
            if gvisit[u] == False:
                rec = self.BFS(u)
                comps.append(len(rec))
                for i in rec:
                  gvisit[i] = True
        return comps

    def is_connected(self) -> bool:
        return self.number_of_components() == 1

